In [1]:
print("hello world")

hello world


In [1]:
import torch
import torchaudio

Okay so we have to write a custom dataset class no biggie, but the project description says to focus on silence and unknown but there is no such labels 

I guess for unknown you can make argument that special words like marvin, sheila, happy can serve as unknown but not sure about silence.

Okay case closed we are supposed to create it ourselves based on background_noise_ directory 

Goal for this notebook is to prepare data. I'm assuming that in /data
directory should be extracted contents of train archive:

- `audio` directory 
- testing_list.txt
- validation_list.txt

I'm gonna create three directories in /data

- `train`
- `test`
- `validation`

im gonna add new class to each: silence